# Linear programming with SciPy and PuLP - RealPython Guide

From RealPython - Hands-On Linear Programming: Optimization With Python   
https://realpython.com/linear-programming-python/#reader-comments

- Linear programming is a set of techniques used in mathematical programming, sometimes called mathematical optimization, to solve systems of linear equations and inequalities while maximizing or minimizing some linear function.     
- Often, when people try to formulate and solve an optimization problem, the first question is whether they can apply linear programming or mixed-integer linear programming.    
- The importance of linear programming, and especially mixed-integer linear programming, has increased over time as computers have gotten more capable, algorithms have improved, and more user-friendly software solutions have become available.



#### Terminology
**Mixed-integer linear programming** is an extension of linear programming. It handles problems in which at least one variable takes a discrete integer rather than a continuous value. Although mixed-integer problems look similar to continuous variable problems at first sight, they offer significant advantages in terms of flexibility and precision.

**Integer variables** are important for properly representing quantities naturally expressed with integers, like the number of airplanes produced or the number of customers served.

**Binary variables** are a particularly important kind of integer variable. It can take only the values zero or one and is useful in making yes-or-no decisions, such as whether a plant should be built or if a machine should be turned on or off. You can also use them to mimic logical constraints.

**Solvers**, It’s worth mentioning that almost all widely used linear programming and mixed-integer linear programming libraries are native to and written in Fortran or C or C++. This is because linear programming requires computationally intensive work with (often large) matrices. Such libraries are called solvers. The Python tools are just wrappers around the solvers.

Several free Python libraries are specialized to interact with linear or mixed-integer linear programming solvers:
- SciPy Optimization and Root Finding
- PuLP
- Pyomo
- CVXOPT   

Some well-known and very powerful commercial and proprietary solutions are 
- Gurobi
- CPLEX
- XPRESS.   

Besides offering flexibility when defining problems and the ability to run various solvers, PuLP is less complicated to use than alternatives like Pyomo or CVXOPT, which require more time and effort to master.    
In this tutorial, you’ll use SciPy and PuLP to define and solve linear programming problems.

## 1. Small/Easy Visual Problem
**Maximize:**
- $ z = x + 2y $

**Subject to:**
- $ 2x + y \leq 20 $   
- $-4x + 5y \leq 10 $   
- $ -x + 2y \geq -2 $   
- $ x \geq 0 $   
- $ y \geq 0 $   

You need to find x and y such that the red, blue, and yellow inequalities, as well as the inequalities x ≥ 0 and y ≥ 0, are satisfied. At the same time, your solution must correspond to the largest possible value of z.

**Decision variables** are the independent variables you need to find. x and y in this case   
**Objective function/Cost function/goal**, the function of the decision variables to be maximized or minimized. In this case z.   
**Inequality constraints**   
**Equality constraints**

We now expand the problem with the additional equality constraint shown in green. If you insert the demand that all values of x must be integers, then you’ll get a mixed-integer linear programming problem, and the set of feasible solutions will change once again:  

<img src="https://files.realpython.com/media/lp-py-fig-1.00f609c97aec.png" width="300"/>   

<img src="https://files.realpython.com/media/lp-py-fig-3.c13d0660ce57.png" width="300"/>    


## 2. Real Resource Allocation Problem

Say that a factory produces four different products, and that the daily produced amount of the first product is x₁, the amount produced of the second product is x₂, and so on. The goal is to determine the profit-maximizing daily production amount for each product, bearing in mind the following conditions:

The profit per unit of product is $20, $12, $40, and $25 for the first, second, third, and fourth product, respectively.

Due to manpower constraints, the total number of units produced per day can’t exceed fifty.

For each unit of the first product, three units of the raw material A are consumed. Each unit of the second product requires two units of the raw material A and one unit of the raw material B. Each unit of the third product needs one unit of A and two units of B. Finally, each unit of the fourth product requires three units of B.

Due to the transportation and storage constraints, the factory can consume up to one hundred units of the raw material A and ninety units of B per day.   
The mathematical model can be defined like this:

**Objective: Maximize Profit**
- $20x_1 + 12x_2 + 40x_3 + 25x_4$

**Subject to:**
- $x_1 + x_2 + x_3 + x_4 \leq 50 (manpower)  $
- $3x_1 + 2x_2 + x_3 \leq 100 (material A)$
- $x_2 + 2x_3 + 3x_4 \leq 90 (material B)$
- $x_1, x_2, x_3, x_4 \geq 0$


## 3. Linear Programming with SciPy
### 3.1 Solving the small visual problem
We will use the SciPy optimization and root-finding library for linear programming.    
To define and solve optimization problems with SciPy, you need to import scipy.optimize.linprog():

In [1]:
# imports
from scipy.optimize import linprog

**linprog() solves only minimization** (not maximization) problems and **doesn’t allow inequality constraints with the greater or equal sign (≥)** (need to flip it).    
 To work around these issues, you need to modify your problem before starting optimization:
- Instead of maximizing z = x + 2y, you can minimize its negative(−z = −x − 2y).
- Instead of having the greater than or equal to sign, you can multiply the yellow inequality by −1 and get the opposite less than or equal to sign (≤).   
After introducing these changes, you get a new system:

**Maximize:**   
- $ -z = -x - 2y $

**Subject to:**   
- $ 2x + y \leq 20 $   
- $ -4x + 5y \leq 10 $   
- $ x - 2y \leq 2 $   
- $ x \geq 0 $   
- $ y \geq 0 $   

This system is equivalent to the original and will have the same solution. The only reason to apply these changes is to overcome the limitations of SciPy related to the problem formulation.

In [2]:
# Objective function: (-z = -x - 2y)
obj = [-1, -2]
#      ─┬  ─┬
#       │   └┤ Coefficient for y
#       └────┤ Coefficient for x

lhs_ineq = [[ 2,  1],  # Red constraint left side
            [-4,  5],  # Blue constraint left side
            [ 1, -2]]  # Yellow constraint left side

rhs_ineq = [20,  # Red constraint right side
            10,  # Blue constraint right side
             2]  # Yellow constraint right side

lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]       # Green constraint right side

The next step is to define the bounds for each variable in the same order as the coefficients. In this case, they’re both between zero and positive infinity.   
This statement is redundant because linprog() takes these bounds (zero to positive infinity) by default.

In [3]:
bnd = [(0, float("inf")),  # Bounds of x
       (0, float("inf"))]  # Bounds of y

Finally, it’s time to optimize and solve your problem of interest. You can do that with **linprog()**.   
- The parameter c refers to the coefficients from the objective function. 
- A_ub and b_ub are related to the coefficients from the left and right sides of the inequality constraints, respectively. 
- Similarly, A_eq and b_eq refer to equality constraints. 
- You can use bounds to provide the lower and upper bounds on the decision variables.

You can use the parameter 'method' to define the linear programming method that you want to use. There are three options:
- method="interior-point" selects the interior-point method. This option is set by default.
- method="revised simplex" selects the revised two-phase simplex method.
- method="simplex" selects the legacy two-phase simplex method.

In [4]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="revised simplex")
opt

C:\Users\torha\AppData\Local\Temp\ipykernel_13208\1019385060.py:1: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -16.818181818181817
       x: [ 7.727e+00  4.545e+00]
     nit: 3

linprog() returns a data structure with these attributes:
- .con is the equality constraints residuals.
- .fun is the objective function value at the optimum (if found).
- .message is the status of the solution.
- .nit is the number of iterations needed to finish the calculation.
- .slack is the values of the slack variables, or the differences between the values of the left and right sides of the constraints.
- .status is an integer between 0 and 4 that shows the status of the solution, such as 0 for when the optimal solution has been found.
- .success is a Boolean that shows whether the optimal solution has been found.
- .x is a NumPy array holding the optimal values of the decision variables.

We can also access these values separately.   
We can also plot the results from these values.

In [5]:
print("the objective function value at the optimum (if found):", round(opt.fun,4))
print("Boolean that shows whether the optimal solution has been found:", opt.success)
print("NumPy array holding the optimal values of the decision variables:", opt.x)

the objective function value at the optimum (if found): -16.8182
Boolean that shows whether the optimal solution has been found: True
NumPy array holding the optimal values of the decision variables: [7.72727273 4.54545455]


If you want to exclude the equality (green) constraint, just drop the parameters A_eq and b_eq from the linprog() call:

In [6]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,
              method="revised simplex")
opt


C:\Users\torha\AppData\Local\Temp\ipykernel_13208\106174338.py:1: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -20.714285714285715
       x: [ 6.429e+00  7.143e+00]
     nit: 2

In [7]:
print("the objective function value at the optimum (if found):", round(opt.fun,4))
print("Boolean that shows whether the optimal solution has been found:", opt.success)
print("NumPy array holding the optimal values of the decision variables:", opt.x)

the objective function value at the optimum (if found): -20.7143
Boolean that shows whether the optimal solution has been found: True
NumPy array holding the optimal values of the decision variables: [6.42857143 7.14285714]


### 3.1 Solving the resource allocation problem
**Objective: Maximize Profit**
- $20x_1 + 12x_2 + 40x_3 + 25x_4$

**Subject to:**
- $x_1 + x_2 + x_3 + x_4 \leq 50 (manpower)  $
- $3x_1 + 2x_2 + x_3 \leq 100 (material A)$
- $x_2 + 2x_3 + 3x_4 \leq 90 (material B)$
- $x_1, x_2, x_3, x_4 \geq 0$

As in the previous example, we need to extract the necessary vectors and matrix from the problem above,    
pass them as the arguments to .linprog(), and get the results:

In [8]:
# Objective function: (-z = -x - 2y)
obj = [-20, -12, -40, -25]

lhs_ineq = [[1, 1, 1, 1],  # Manpower
            [3, 2, 1, 0],  # Material A
            [0, 1, 2, 3]]  # Material B

rhs_ineq = [ 50,  # Manpower
            100,  # Material A
             90]  # Material B

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, method="revised simplex")
opt

C:\Users\torha\AppData\Local\Temp\ipykernel_13208\1625264034.py:12: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, method="revised simplex")


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -1900.0
       x: [ 5.000e+00  0.000e+00  4.500e+01  0.000e+00]
     nit: 2

In [9]:
print("the objective function value at the optimum (if found):", round(opt.fun,4))
print("Boolean that shows whether the optimal solution has been found:", opt.success)
print("NumPy array holding the optimal values of the decision variables:", opt.x)

the objective function value at the optimum (if found): -1900.0
Boolean that shows whether the optimal solution has been found: True
NumPy array holding the optimal values of the decision variables: [ 5.  0. 45.  0.]


In [10]:
# Can treat it like a dictionary
for a,b in opt.items():
    print(a, ":", b)

x : [ 5.  0. 45.  0.]
fun : -1900.0
slack : [ 0. 40.  0.]
con : []
status : 0
message : Optimization terminated successfully.
nit : 2
success : True


The result tells you that the maximal profit is 1900 and corresponds to x₁ = 5 and x₃ = 45.    
It’s not profitable to produce the second and fourth products under the given conditions.    
You can draw several interesting conclusions here:

- The third product brings the largest profit per unit, so the factory will produce it the most.
- The first slack is 0, which means that the values of the left and right sides of the manpower (first) constraint are the same.   
The factory produces 50 units per day, and that’s its full capacity.
- The second slack is 40 because the factory consumes 60 units of raw material A (15 units for the first product plus 45 for the third) out of a potential 100 units.
- The third slack is 0, which means that the factory consumes all 90 units of the raw material B. This entire amount is consumed for the third product.    
That’s why the factory can’t produce the second or fourth product at all and can’t produce more than 45 units of the third product. It lacks the raw material B.  

opt.status is 0 and opt.success is True, indicating that the optimization problem was successfully solved with the optimal feasible solution.

#### Summary of SciPy's capabilities for solving Linear Problems
SciPy’s linear programming capabilities are useful mainly for smaller problems. For larger and more complex problems, you might find other libraries more suitable for the following reasons:

- SciPy can’t run various external solvers.
- SciPy can’t work with integer decision variables.
- SciPy doesn’t provide classes or functions that facilitate model building. To define arrays and matrices is tedious/error-prone task for large problems.
- SciPy doesn’t allow you to define maximization problems directly. You must convert them to minimization problems.
- SciPy doesn’t allow you to define constraints using the greater-than-or-equal-to sign directly. You must use the less-than-or-equal-to instead.

Fortunately, the Python ecosystem offers several alternative solutions for linear programming that are very useful for larger problems.   
One of them is **PuLP**, which you’ll see in action in the next section.

## 4. Linear Programming with PuLP
### 4.1 Solving the small visual problem
*"PuLP is an LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems."*

PuLP has a more convenient linear programming API than SciPy. You don’t have to mathematically modify your problem or use vectors and matrices. Everything is cleaner and less prone to errors.

As usual, you start by importing what you need:

In [11]:
# imports
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

**Small/Easy Visual Problem**   
**Maximize:**
- $ z = x + 2y $

**Subject to:**
- $ 2x + y \leq 20 $   
- $-4x + 5y \leq 10 $   
- $ -x + 2y \geq -2 $   
- $ x \geq 0 $   
- $ y \geq 0 $   

The first step is to initialize an instance of [LpProblem](https://www.coin-or.org/PuLP/pulp.html#pulp.LpProblem) to represent your model:

**Parameters**:	
- name – name of the problem used in the output .lp file
- sense – of the LP problem objective. Either LpMinimize ('1'; default) or LpMaximize('-1').

**Returns**:	
- An LP Problem

In [12]:
# Create the model - we chose maximization for our problem
model = LpProblem(name="small-problem", sense=LpMaximize)

Once that you have the model, you can define the decision variables as instances of the [LpVariable](https://www.coin-or.org/PuLP/pulp.html#pulp.LpVariable) class:

Parameters:	
- name – The name of the variable used in the output .lp file
- lowbound – The lower bound on this variable’s range. Default is negative infinity
- upBound – The upper bound on this variable’s range. Default is positive infinity
- cat – The category this variable is in, Integer, Binary or Continuous(default)
- e – Used for column based modelling: relates to the variable’s existence in the objective function and constraints


In [13]:
# Initialize the decision variables
x = LpVariable(name="x", lowBound=0) # upBound is positive infinity by default
y = LpVariable(name="y", lowBound=0) # upBound is positive infinity by default

The optional parameter cat defines the category of a decision variable. If you’re working with continuous variables, then you can use the default value "Continuous".   
You can use the variables x and y to create other PuLP objects that represent linear expressions and constraints:

In [14]:
# just examples here
expression = 2 * x + 4 * y
print(type(expression))


constraint = 2 * x + 4 * y >= 8
print(type(constraint))

<class 'pulp.pulp.LpAffineExpression'>
<class 'pulp.pulp.LpConstraint'>


When you multiply a decision variable with a scalar or build a linear combination of multiple decision variables, you get an instance of pulp.LpAffineExpression that represents a linear expression.

> Note:
> You can add or subtract variables or expressions, and you can multiply them with constants because PuLP classes implement some of the Python special methods that emulate numeric types like __add__(), __sub__(), and __mul__().   
> These methods are used to customize the behavior of operators like +, -, and *.

Similarly, you can combine linear expressions, variables, and scalars with the operators ==, <=, or >= to get instances of pulp.LpConstraint that represent the linear constraints of your model.

> Note: 
> It’s also possible to build constraints with the rich comparison methods .__eq__(), .__le__(), and .__ge__() that define the behavior of the operators ==, <=, and >=.

Having this in mind, the next step is to create the constraints and objective function as well as to assign them to your model. You don’t need to create lists or matrices. Just write Python expressions and use the += operator to append them to the model:

In [15]:
# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

In the above code, you define tuples that hold the constraints and their names. LpProblem allows you to add constraints to a model by specifying them as tuples.    
The first element is a LpConstraint instance. The second element is a human-readable name for that constraint.

Setting the objective function is very similar:

In [16]:
# Add the objective function to the model
obj_func = x + 2 * y
model += obj_func

# Can also use shorter notation like this
# model += x + 2 * y

Now you have the objective function added and the model defined.

For larger problems, it’s often more convenient to use **lpSum()** with a list or other sequence than to repeat the + operator.   
For example, you could add the objective function to the model with this statement, which produce the same result as the previous statement.

```python
# Add the objective function to the model
model += lpSum([x, 2 * y])
```

You can now see the full definition of this model:

In [17]:
model

small-problem:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
red_constraint: 2 x + y <= 20

blue_constraint: 4 x - 5 y >= -10

yellow_constraint: - x + 2 y >= -2

green_constraint: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous

The string representation of the model contains all relevant data: the variables, constraints, objective, and their names.

Finally, you’re ready to solve the problem. You can do that by calling .solve() on your model object. If you want to use the default solver (CBC), then you don’t need to pass any arguments:

In [18]:
# Solve the problem
status = model.solve()
status

1

.solve() calls the underlying solver, modifies the model object, and returns the integer status of the solution, which will be 1 if the optimum is found.   
 For the rest of the status codes, see [LpStatus](https://www.coin-or.org/PuLP/constants.html#pulp.constants.LpStatus).

 
| LpStatus key         | string value  | numerical value |
|----------------------|---------------|-----------------|
| LpStatusOptimal      | "Optimal"     | 1               |
| LpStatusNotSolved    | "Not Solved"  | 0               |
| LpStatusInfeasible   | "Infeasible"  | -1              |
| LpStatusUnbounded    | "Unbounded"   | -2              |
| LpStatusUndefined    | "Undefined"   | -3              |

You can get the optimization results as the attributes of model. The function value() and the corresponding method .value() return the actual values of the attributes:

In [19]:
print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 16.8181817
x: 7.7272727
y: 4.5454545
red_constraint: -9.99999993922529e-08
blue_constraint: 18.181818300000003
yellow_constraint: 3.3636362999999996
green_constraint: -2.0000000233721948e-07


model.objective holds the value of the objective function, model.constraints contains the values of the slack variables, and the objects x and y have the optimal values of the decision variables. model.variables() returns a list with the decision variables:

In [20]:
print(model.variables())
print(model.variables()[0] is x)
print(model.variables()[1] is y)

[x, y]
True
True


As you can see, this list contains the exact objects that are created with the constructor of LpVariable.

The results are approximately the same as the ones you got with SciPy.

> Note: Be careful with the method .solve()—it changes the state of the objects x and y!

You can see which solver was used by calling .solver:

In [21]:
model.solver

The output informs you that the solver is CBC. You didn’t specify a solver, so PuLP called the default one.

If you want to run a different solver, then you can specify it as an argument of .solve(). For example, if you want to use GLPK and already have it installed, then you can use ```solver=GLPK(msg=False)``` in the last line. Keep in mind that you’ll also need to import it.   
When you have GLPK imported, you can use it inside .solve():

In [22]:
from pulp import GLPK

# Solve the problem
status = model.solve(solver=GLPK(msg=False))
status

1

The msg parameter is used to display information from the solver. msg=False disables showing this information. If you want to include the information, then just omit msg or set msg=True.

Your model is defined and solved, so you can inspect the results the same way you did in the previous case:

In [23]:
print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 16.81817
x: 7.72727
y: 4.54545
red_constraint: -1.0000000000509601e-05
blue_constraint: 18.181830000000005
yellow_constraint: 3.3636299999999997
green_constraint: -2.000000000279556e-05


We got practically the same result with GLPK as you did with SciPy and CBC.

Let’s peek and see which solver was used this time:

In [24]:
model.solver

As you defined above with the highlighted statement model.solve(solver=GLPK(msg=False)), the solver is GLPK.

You can also use PuLP to solve mixed-integer linear programming problems. To define an integer or binary variable, just pass cat="Integer" or cat="Binary" to LpVariable. Everything else remains the same:

In [25]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables: x is integer, y is continuous
x = LpVariable(name="x", lowBound=0, cat="Integer") # NEW CHANGE
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve()

In this example, you have one integer variable and get different results from before:

In [26]:
print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")


for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

model.solver

status: 1, Optimal
objective: 15.8
x: 7.0
y: 4.4
red_constraint: -1.5999999999999996
blue_constraint: 16.0
yellow_constraint: 3.8000000000000007
green_constraint: 0.0


Now x is an integer, as specified in the model. (Technically it holds a float value with zero after the decimal point.) This fact changes the whole solution.   
Let’s show this on the graph:    
As you can see, the optimal solution is the rightmost green point on the gray background. This is the feasible solution with the largest values of both x and y, giving it the maximal objective function value.

GLPK is capable of solving such problems as well.


### 4.2 Solving the resource allocation problem
The approach for defining and solving the problem is the same as in the previous example:

**Objective: Maximize Profit**
- $20x_1 + 12x_2 + 40x_3 + 25x_4$

**Subject to:**
- $x_1 + x_2 + x_3 + x_4 \leq 50 (manpower)  $
- $3x_1 + 2x_2 + x_3 \leq 100 (material A)$
- $x_2 + 2x_3 + 3x_4 \leq 90 (material B)$
- $x_1, x_2, x_3, x_4 \geq 0$

In this case, you use the dictionary x to store all decision variables. This approach is convenient because dictionaries can store the names or indices of decision variables as keys and the corresponding LpVariable objects as values. Lists or tuples of LpVariable instances can be useful as well.

The code above produces the following result:

In [29]:
# Define the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

# Set the objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 1900.0
x1: 5.0
x2: 0.0
x3: 45.0
x4: 0.0
manpower: 0.0
material_a: -40.0
material_b: 0.0


As you can see, the solution is consistent with the one obtained using SciPy. The most profitable solution is to produce 5.0 units of the first product and 45.0 units of the third product per day.

Let’s make this problem more complicated and interesting. Say the factory can’t produce the first and third products in parallel due to a machinery issue. What’s the most profitable solution in this case?

Now you have another logical constraint: if x₁ is positive, then x₃ must be zero and vice versa. This is where binary decision variables are very useful. You’ll use two binary decision variables, y₁ and y₃, that’ll denote if the first or third products are generated at all:

In [30]:
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
y = {i: LpVariable(name=f"y{i}", cat="Binary") for i in (1, 3)}    # NEW CHANGE

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

M = 100                                      # NEW CHANGE
model += (x[1] <= y[1] * M, "x1_constraint") # NEW CHANGE
model += (x[3] <= y[3] * M, "x3_constraint") # NEW CHANGE
model += (y[1] + y[3] <= 1, "y_constraint")  # NEW CHANGE

# Set objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 1800.0
x1: 0.0
x2: 0.0
x3: 45.0
x4: 0.0
y1: 0.0
y3: 1.0
manpower: -5.0
material_a: -55.0
material_b: 0.0
x1_constraint: 0.0
x3_constraint: -55.0
y_constraint: 0.0


The code was very similar to the previous example except for the highlighted lines. Here are the differences:

- Line 5 defines the binary decision variables y[1] and y[3] held in the dictionary y.
- Line 12 defines an arbitrarily large number M. The value 100 is large enough in this case because you can’t have more than 100 units per day.
- Line 13 says that if y[1] is zero, then x[1] must be zero, else it can be any non-negative number.
- Line 14 says that if y[3] is zero, then x[3] must be zero, else it can be any non-negative number.
- Line 15 says that either y[1] or y[3] is zero (or both are), so either x[1] or x[3] must be zero as well.

Final result:   
It turns out that the optimal approach is to exclude the first product and to produce only the third one.

## 5. Summary

#### To summarize, we have learned how to:

- Define a model that represents your problem
- Create a Python program for optimization
- Run the optimization program to find the solution to the problem
- Retrieve the result of optimization

We used SciPy with its own solver as well as PuLP with CBC and GLPK, but also learned that there are many other linear programming solvers and Python wrappers.